In [1]:
import pandas as pd 
import psycopg2
import os

In [2]:
DB_CONFIG = {
    'host' : 'db',
    'database' : 'verso_database',
    'user' : 'root',
    'password' : 'versosql'
}

In [12]:
conn = psycopg2.connect(**DB_CONFIG) 
cursor = conn.cursor()
cursor.execute("SELECT spot_table.Timestamp, spot_table.spot_price, pv_prod_table.pv_prod FROM (SELECT delivery_start AS Timestamp, price AS spot_price FROM elec_day_ahead_market WHERE tenor = 'PT1H') AS spot_table LEFT JOIN (SELECT date_trunc('hour', prod_start) AS Timestamp, AVG(quantity) AS pv_prod FROM production_per_type WHERE tenor = 'PT1H' AND production_type = 'SOLAR' GROUP BY Timestamp) AS pv_prod_table ON spot_table.Timestamp = pv_prod_table.Timestamp ORDER BY spot_table.Timestamp")
results = cursor.fetchall()

In [14]:
df = pd.DataFrame(data=results, columns=[d.name for d in cursor.description])

## Consumption

In [4]:
path = '/frontend/data/Consumption'
os.makedirs(path, exist_ok=True)

In [23]:
conn = psycopg2.connect(**DB_CONFIG) 
cursor = conn.cursor()
cursor.execute("SELECT cons_start, quantity FROM consumption WHERE tenor = 'PT15M' AND curve_type = 'REALISED' ORDER BY cons_start")
results = cursor.fetchall()
cons_df = pd.DataFrame(data=results, columns=['Timestamp', 'Cons'])
cons_df.Cons = cons_df.Cons.astype(float)
cons_df.set_index('Timestamp', inplace = True)

In [24]:
cons_df = cons_df.resample('h').mean()
groups = cons_df.groupby(cons_df.index.year)
for year in range(2016, 2025):
    groups.get_group(year).to_excel(f'{path}/{year}.xlsx')

## PV + Capacity factor

### PV

In [89]:
path = '/frontend/data/PV'
os.makedirs(path, exist_ok=True)

In [90]:
conn = psycopg2.connect(**DB_CONFIG) 
cursor = conn.cursor()
cursor.execute("SELECT date_trunc('hour', prod_start) AS Timestamp, AVG(quantity) AS value FROM production_per_type WHERE tenor = 'PT1H' AND production_type = 'SOLAR' GROUP BY Timestamp ORDER BY Timestamp")
results = cursor.fetchall()
prod_df = pd.DataFrame(data=results, columns=['Timestamp', 'PV'])
prod_df.PV = prod_df.PV.astype(float)
prod_df.set_index('Timestamp', inplace = True)

In [91]:
prod_df = prod_df.resample('h').interpolate('time')

In [92]:
groups = prod_df.groupby(prod_df.index.year)
for year in range(2015, 2025):
    groups.get_group(year).to_excel(f'{path}/{year}.xlsx')

### Capacity factor

In [93]:
path = '/frontend/data/PV_CF'
os.makedirs(path, exist_ok=True)

In [94]:
prod_df = prod_df.tz_localize('UTC').tz_convert('CET')
prod_df['Year'] = prod_df.index.year
prod_df.reset_index(inplace=True)

In [95]:
conn = psycopg2.connect(**DB_CONFIG) 
cursor = conn.cursor()
cursor.execute("SELECT capa_start, quantity FROM installed_capacities WHERE capacities_type = 'SOLAR' ORDER BY capa_start")
results = cursor.fetchall()
capa_df = pd.DataFrame(data=results, columns=['start', 'PV_capa'])
capa_df.PV_capa = capa_df.PV_capa.astype(float)
capa_df.start = pd.to_datetime(capa_df.start, utc=True).dt.tz_convert('CET')
capa_df['Year'] = capa_df.start.dt.year

In [96]:
cf_df = pd.merge(left=prod_df, right=capa_df, left_on='Year', right_on='Year').set_index('Timestamp')[['PV', 'PV_capa']]
cf_df['PV_cf'] = cf_df.PV / cf_df.PV_capa
cf_df = cf_df[['PV_cf']].tz_convert('UTC').tz_localize(None)

In [97]:
groups = cf_df.groupby(cf_df.index.year)
for year in range(2015, 2024):
    groups.get_group(year).to_excel(f'{path}/{year}.xlsx')

## Wind + Capacity factor

### Wind

In [103]:
path = '/frontend/data/Wind'
os.makedirs(path, exist_ok=True)

In [104]:
conn = psycopg2.connect(**DB_CONFIG) 
cursor = conn.cursor()
cursor.execute("SELECT date_trunc('hour', prod_start) AS Timestamp, "
               "AVG(value) as Prod "
               "FROM (SELECT prod_start, SUM(quantity) AS value "
               "FROM production_per_type "
               "WHERE tenor = 'PT1H' AND "
               "production_type IN ('WIND_OFFSHORE', 'WIND_ONSHORE') "
               "GROUP BY prod_start) "
               "GROUP BY Timestamp ORDER BY Timestamp")
results = cursor.fetchall()
prod_df = pd.DataFrame(data=results, columns=['Timestamp', 'Wind'])
prod_df.Wind = prod_df.Wind.astype(float)
prod_df.set_index('Timestamp', inplace = True)

In [105]:
prod_df = prod_df.resample('h').interpolate('time')

In [106]:
groups = prod_df.groupby(prod_df.index.year)
for year in range(2015, 2025):
    groups.get_group(year).to_excel(f'{path}/{year}.xlsx')

### Capacity factor

In [107]:
path = '/frontend/data/Wind_CF'
os.makedirs(path, exist_ok=True)

In [109]:
prod_df = prod_df.tz_localize('UTC').tz_convert('CET')
prod_df['Year'] = prod_df.index.year
prod_df.reset_index(inplace=True)

In [110]:
conn = psycopg2.connect(**DB_CONFIG) 
cursor = conn.cursor()
cursor.execute("SELECT capa_start, SUM(quantity) "
               "FROM installed_capacities "
               "WHERE capacities_type IN ('WIND_OFFSHORE', 'WIND_ONSHORE') "
               "GROUP BY capa_start ORDER BY capa_start")
results = cursor.fetchall()
capa_df = pd.DataFrame(data=results, columns=['start', 'Wind_capa'])
capa_df.Wind_capa = capa_df.Wind_capa.astype(float)
capa_df.start = pd.to_datetime(capa_df.start, utc=True).dt.tz_convert('CET')
capa_df['Year'] = capa_df.start.dt.year

In [113]:
cf_df = pd.merge(left=prod_df, right=capa_df, left_on='Year', right_on='Year').set_index('Timestamp')[['Wind', 'Wind_capa']]
cf_df['Wind_cf'] = cf_df.Wind / cf_df.Wind_capa
cf_df = cf_df[['Wind_cf']].tz_convert('UTC').tz_localize(None)

In [114]:
groups = cf_df.groupby(cf_df.index.year)
for year in range(2015, 2024):
    groups.get_group(year).to_excel(f'{path}/{year}.xlsx')